In [1]:
import pandas as pd
import geopandas as gp
import os


In [2]:
bay_county_names = ["Alameda", "ContraCosta", "Sonoma", "Solano", "SanMateo", "SantaClara", "SanFrancisco", "Marin","Napa"] 
analysis_years = ["2008", "2009","2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017"]
slump_years = ["2008", "2009","2010", "2011", "2012"]
recovery_years = ["2013", "2014", "2015", "2016", "2017"]

#### CT- Place crosswalk from Missouri geocorr 2014

In [4]:
crosswalk = pd.read_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/reference/geocorr2014.csv")
crosswalk.head()

,County code,Tract,State code,Place code,State abbreviation,County name,Place name,Total housing units (2010),tract to placefp allocation factor
0,6013,3400.02,6,135,CA,Contra Costa CA,"Acalanes Ridge CDP, CA",457,0.151
1,6001,4271.00,6,562,CA,Alameda CA,"Alameda city, CA",1469,1
2,6001,4272.00,6,562,CA,Alameda CA,"Alameda city, CA",1730,1
3,6001,4273.00,6,562,CA,Alameda CA,"Alameda city, CA",2212,1
4,6001,4276.00,6,562,CA,Alameda CA,"Alameda city, CA",1881,1


#### Merge the file with all the CTs in the 9 counties for each year with the crosswalk, and replace the empty place names with "unincorporated non CDPs". Exports a csv for each year that includes the place designation of each CT

In [5]:
for year in analysis_years:
    file = pd.read_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/CTs_geo_data/all_bay_concat_"+year+".csv")
    merged = file.merge(crosswalk, how= 'left', left_on="Tract", right_on="Tract")
    merged["Place name"].replace(to_replace=' ', value='unincorporated non CDP', inplace=True)
    #merged.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/place_parsed/allbay_place_geolocated_"+str(year)+"_.csv")
    #these will be dataframes with place assigned to them, I will want to concatenate them with the years in the
    #defined range and then groupby average, so for now I should export them as individual years
merged

,Tract,Year,County,type,CRA Eligible,"% HI borrower, LI tract","# HI borrower, LI tract","% HI borrower, HI tract","# HI borrower, HI tract","% LI borrower, LI tract",...,FUNCSTAT00,geometry,County code,State code,Place code,State abbreviation,County name,Place name,Total housing units (2010),tract to placefp allocation factor
0,2501.03,2017,Solano County,mid,not eligible,0.000000,0,0.867470,72,0.0,...,NaN,NaN,6095,6,81666,CA,Solano CA,"Vallejo city, CA",1668,1
1,2501.04,2017,Solano County,high,not eligible,0.000000,0,0.842105,32,0.0,...,NaN,NaN,6095,6,81666,CA,Solano CA,"Vallejo city, CA",853,1
2,2501.05,2017,Solano County,mid,not eligible,0.000000,0,0.879433,124,0.0,...,NaN,NaN,6095,6,81666,CA,Solano CA,"Vallejo city, CA",2258,1
3,2501.06,2017,Solano County,high,not eligible,0.000000,0,0.978571,137,0.0,...,NaN,NaN,6095,6,81666,CA,Solano CA,"Vallejo city, CA",1440,0.999
4,2501.06,2017,Solano County,high,not eligible,0.000000,0,0.978571,137,0.0,...,NaN,NaN,6095,6,99999,CA,Solano CA,unincorporated non CDP,2,0.001
5,2502.00,2017,Solano County,mod,eligible,0.906977,78,0.000000,0,0.0,...,S,"POLYGON ((-122.201761 38.099908, -122.202042 3...",6095,6,81666,CA,Solano CA,"Vallejo city, CA",1474,0.955
6,2502.00,2017,Solano County,mod,eligible,0.906977,78,0.000000,0,0.0,...,S,"POLYGON ((-122.201761 38.099908, -122.202042 3...",6095,6,99999,CA,Solano CA,unincorporated non CDP,70,0.045
7,2503.00,2017,Solano County,mod,eligible,0.648000,81,0.000000,0,0.0,...,S,"POLYGON ((-122.217936 38.099869, -122.218133 3...",6095,6,81666,CA,Solano CA,"Vallejo city, CA",1221,1
8,2504.00,2017,Solano County,mod,eligible,0.604938,49,0.000000,0,0.0,...,S,"POLYGON ((-122.229733 38.096133, -122.229731 3...",6095,6,81666,CA,Solano CA,"Vallejo city, CA",1248,0.974
9,2504.00,2017,Solano County,mod,eligible,0.604938,49,0.000000,0,0.0,...,S,"POLYGON ((-122.229733 38.096133, -122.229731 3...",6095,6,99999,CA,Solano CA,unincorporated non CDP,33,0.026


#### Append the dataframe for all the years in each of the ranges "slump" and "recovery" to create a dataframe for each range

In [7]:
path = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/place_parsed/"
for year in analysis_years:
    slump_csvs = []
    recovery_csvs = []
    slump_file_names = [f for f in os.listdir(path) 
                    if f[-3:] == "csv"
                    and f.split('_')[3] in slump_years]
    for file in slump_file_names:   
        df = pd.read_csv(path+file)
        slump_csvs.append(df)
    slump = pd.concat(slump_csvs, axis=0)
    recovery_file_names = [f for f in os.listdir(path) 
                    if f[-3:] == "csv"
                    and f.split('_')[3] in recovery_years]
    for file in recovery_file_names:
        df = pd.read_csv(path + file)
        recovery_csvs.append(df)
    recovery = pd.concat(recovery_csvs, axis=0)




In [9]:
recovery

,Unnamed: 0,Tract,Year,County,type,CRA Eligible,"% HI borrower, LI tract","# HI borrower, LI tract","% HI borrower, HI tract","# HI borrower, HI tract",...,FUNCSTAT00,geometry,County code,State code,Place code,State abbreviation,County name,Place name,Total housing units (2010),tract to placefp allocation factor
0,0,2501.03,2013,Solano County,mid,not eligible,0.000000,0,0.705128,55,...,NaN,NaN,6095,6,81666,CA,Solano CA,"Vallejo city, CA",1668,1
1,1,2501.04,2013,Solano County,high,not eligible,0.000000,0,0.700000,28,...,NaN,NaN,6095,6,81666,CA,Solano CA,"Vallejo city, CA",853,1
2,2,2501.05,2013,Solano County,high,not eligible,0.000000,0,0.878571,123,...,NaN,NaN,6095,6,81666,CA,Solano CA,"Vallejo city, CA",2258,1
3,3,2501.06,2013,Solano County,high,not eligible,0.000000,0,0.866667,104,...,NaN,NaN,6095,6,81666,CA,Solano CA,"Vallejo city, CA",1440,0.999
4,4,2501.06,2013,Solano County,high,not eligible,0.000000,0,0.866667,104,...,NaN,NaN,6095,6,99999,CA,Solano CA,unincorporated non CDP,2,0.001
5,5,2502.00,2013,Solano County,mod,eligible,0.568966,33,0.000000,0,...,S,"POLYGON ((-122.201761 38.099908, -122.202042 3...",6095,6,81666,CA,Solano CA,"Vallejo city, CA",1474,0.955
6,6,2502.00,2013,Solano County,mod,eligible,0.568966,33,0.000000,0,...,S,"POLYGON ((-122.201761 38.099908, -122.202042 3...",6095,6,99999,CA,Solano CA,unincorporated non CDP,70,0.045
7,7,2503.00,2013,Solano County,mod,eligible,0.470588,16,0.000000,0,...,S,"POLYGON ((-122.217936 38.099869, -122.218133 3...",6095,6,81666,CA,Solano CA,"Vallejo city, CA",1221,1
8,8,2504.00,2013,Solano County,mid,not eligible,0.000000,0,0.322034,19,...,S,"POLYGON ((-122.229733 38.096133, -122.229731 3...",6095,6,81666,CA,Solano CA,"Vallejo city, CA",1248,0.974
9,9,2504.00,2013,Solano County,mid,not eligible,0.000000,0,0.322034,19,...,S,"POLYGON ((-122.229733 38.096133, -122.229731 3...",6095,6,99999,CA,Solano CA,unincorporated non CDP,33,0.026


#### Find the mean of the number and percentage of each type of loan within the range 

In [50]:
recovery_place_df = recovery.groupby("Place Name").agg('mean')[['% HI borrower, LI tract', '# HI borrower, LI tract',
       '% HI borrower, HI tract', '# HI borrower, HI tract',
       '% LI borrower, LI tract', '# LI borrower, LI tract',
       '%LI borrower, HI tract', '# LI borrower, HI tract', 'STATEFP00',
       'COUNTYFP00', 'FIPS state']]
slump_place_df = slump.groupby("Place Name").agg('mean')[['% HI borrower, LI tract', '# HI borrower, LI tract',
       '% HI borrower, HI tract', '# HI borrower, HI tract',
       '% LI borrower, LI tract', '# LI borrower, LI tract',
       '%LI borrower, HI tract', '# LI borrower, HI tract', 'STATEFP00',
       'COUNTYFP00', 'FIPS state']]
slump_place_df["Period"] = "slump 2008-2012"
recovery_place_df["Period"] = "recovery 2013-2017"

In [ ]:
recovery_place_df.to_csv(path+ "recovery_place_concat.csv")
slump_place_df.to_csv(path+"slump_place_concat.csv")

#### Geocode the place files by using the census shapefiles

In [1]:
#need to geocode the recovery and slump aggregated place files using place census shapefiles
places = gp.read_file("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/raw_shapefiles/pl06_d00_shp/pl06_d00.shp")
places["Merge Name"] = places["NAME"] + " " +places["LSAD_TRANS"]


In [24]:
recovery_csv = pd.read_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/place_parsed/recovery_place_concat.csv")
slump_csv = pd.read_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/place_parsed/slump_place_concat.csv")

In [41]:
slump_geo = slump_csv.merge(places, how='left', left_on='Place Name', right_on='Merge Name').dropna()

recovery_geo = recovery_csv.merge(places, how='left', left_on='Place Name', right_on='Merge Name').dropna()


In [42]:
slump_geo_csv = slump_geo.to_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/place_geo/slump_geo_parsed.csv')
recovery_geo_csv = recovery_geo.to_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/place_geo/recoveru_geo_parsed.csv')